# Get members of groups based on Github repositories
Used for finding out exercise class membership 

In [1]:
from github import Github 
import sys
import datetime
import pandas as pd
import numpy as np
import random
from collections import Counter

In [2]:
# Load personal access token
with open("/Users/jzk870/Documents/NumericalMethods - Harddisk/token/git_access_token.txt", mode = "r") as file:
    token = file.read()

year = "2022"    
class_name = "projects-" + year

### Load all repositories in this year's class room

In [3]:
# a. Access github through access token. This one has been destroyed by Github for safety reasons.
gh = Github(token)
org = gh.get_organization('NumEconCopenhagen')
all_repos = org.get_repos()

# b. Locate all repositories in current class
disregard_repos = ["projects-2022-slet_mig_senere", "projects-2022-crashtest", "projects-2022-team-wmutfmu", 
                   "team-wmutfmu", "projects-2022-guccimane1996", "projects-2022-marko_og_david"]

current_class = [repo.name for repo in all_repos if (class_name in repo.name) & (repo.name not in disregard_repos)]

In [4]:
class_repos = []
groups = {}
admins = ["AndersMunkN", "JeppeDruedahl", "ChristianLangholzCarstensen"]
students = []

for repo in all_repos:
    if repo.name not in current_class:
        continue
    
    # Get the set of collaborators for a repo - remove organization admins
    collaborators = repo.get_collaborators().get_page(0)
    members = [c.login for c in collaborators if c.login not in admins]
    
    # Add members to dictionary and to list of all students
    grp_name = repo.name.removeprefix(class_name+"-")  
    groups[grp_name] = members
    
    students.extend(members)
    

In [5]:
# Make a pandas data frame out of groups and members and print to xlsx
gd = {k:pd.Series(v) for k,v in groups.items()}
groups_df = pd.DataFrame(gd) 

groups_df = groups_df.T

groups_df.rename(columns = {0:'Member1',1:'Member2',2:'Member3',3:'Member4'}, inplace=True)

del groups_df['Member4'] 

groups_df.fillna('',inplace=True)

groups_df.sort_index(inplace=True)

groups_df['ExerciseClass'] = ''

In [6]:
groups_df.reset_index(inplace=True)
groups_df.rename(columns={'index':'Group'}, inplace=True)
groups_df.head()

,Group,Member1,Member2,Member3,ExerciseClass
0,2-brown-1-white,dmz246,dmx963,,
1,21,A5k3r,,,
2,2h2h,Christine-Bach,kasperloewer,,
3,alberte-og-eva,albertehoeg,czq776,,
4,amanda-oliver,amandachris97,qcn966,,


In [14]:
groups_df.to_excel('Groups-And-Exercise-Classes.xlsx', sheet_name='Groups', index = False)